In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Note: All datasets used in this notebook were subset from the original tables.

In [183]:
path = '/home/jewelle/data_bootcamp/'
flights_df =  pd.read_csv(path +'flights_sub.csv')

In [184]:
flights_df.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,...,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,first_dep_time,total_add_gtime,longest_add_gtime,no_name
0,0,2019-03-07,DL,DL,DL,1251,DL,N344DN,1251,10397,...,594.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019-03-07,DL,DL,DL,1252,DL,N314NB,1252,13198,...,1363.0,15.0,0.0,8.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2,2019-03-07,DL,DL,DL,1253,DL,N332NB,1253,14492,...,2354.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2019-03-07,DL,DL,DL,1254,DL,N535US,1254,10397,...,2182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2019-03-07,DL,DL,DL,1255,DL,N393DA,1255,14570,...,422.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [185]:
flights_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [186]:
flight_df.dtypes

Unnamed: 0               int64
fl_date                 object
mkt_unique_carrier      object
branded_code_share      object
mkt_carrier             object
mkt_carrier_fl_num       int64
op_unique_carrier       object
tail_num                object
op_carrier_fl_num        int64
origin_airport_id        int64
origin                  object
origin_city_name        object
dest_airport_id          int64
dest                    object
dest_city_name          object
crs_dep_time             int64
dep_time               float64
dep_delay              float64
taxi_out               float64
wheels_off             float64
wheels_on              float64
taxi_in                float64
crs_arr_time             int64
arr_time               float64
arr_delay              float64
cancelled              float64
cancellation_code       object
diverted               float64
dup                     object
crs_elapsed_time       float64
actual_elapsed_time    float64
air_time               float64
flights 

Regresssion: target variable ARR_DELAY, difference in minutes between scheduled and actual arrival time

Multi-class: CARRIER_DELAY, WEATHER_DELAY, NAS_DELAY, SECURITY_DELAY, LATE_AIRCRAFT_DELAY, predict type of delay

Binary: CANCELLED, predict cancelled flights

In [187]:
#check target variable columns
flights_df.loc[(flights_df['carrier_delay'].notna()), ['carrier_delay',
                                                       'weather_delay',
                                                       'nas_delay',
                                                       'security_delay',
                                                       'late_aircraft_delay',
                                                       'dep_delay',
                                                       'arr_delay', ]]

#all inside flights table

,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,dep_delay,arr_delay
1,15.0,0.0,8.0,0.0,0.0,15.0,23.0
7,5.0,0.0,0.0,0.0,26.0,54.0,31.0
44,76.0,0.0,0.0,0.0,0.0,83.0,76.0
45,1.0,0.0,0.0,0.0,52.0,76.0,53.0
73,35.0,0.0,1.0,0.0,0.0,35.0,36.0
...,...,...,...,...,...,...,...
7968,0.0,0.0,37.0,0.0,0.0,45.0,37.0
7988,6.0,0.0,21.0,0.0,0.0,6.0,27.0
7989,0.0,0.0,16.0,0.0,6.0,17.0,22.0
7993,0.0,0.0,30.0,0.0,0.0,-1.0,30.0


In [188]:
#check for missing values
total = flights_df.isnull().sum().sort_values(ascending=False)
percent = (flights_df.isnull().sum()/flights_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

In [189]:
missing_data.head(20)

,Total,Percent
no_name,8000,1.000000
total_add_gtime,7952,0.994000
first_dep_time,7952,0.994000
longest_add_gtime,7952,0.994000
cancellation_code,7915,0.989375
late_aircraft_delay,6550,0.818750
security_delay,6550,0.818750
nas_delay,6550,0.818750
weather_delay,6550,0.818750
carrier_delay,6550,0.818750


In [190]:
#delay columns have >80% missing data

In [191]:
#drop columns with >82% missing data
drop_columns = missing_data[missing_data['Percent'] > 0.82].index.tolist()
flights_df.drop(drop_columns, axis=1, inplace=True)

In [192]:
flights_df.head(5)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019-03-07,DL,DL,DL,1251,DL,N344DN,1251,10397,ATL,...,109.0,116.0,79.0,1.0,594.0,NaN,NaN,NaN,NaN,NaN
1,2019-03-07,DL,DL,DL,1252,DL,N314NB,1252,13198,MCI,...,225.0,233.0,218.0,1.0,1363.0,15.0,0.0,8.0,0.0,0.0
2,2019-03-07,DL,DL,DL,1253,DL,N332NB,1253,14492,RDU,...,377.0,379.0,344.0,1.0,2354.0,NaN,NaN,NaN,NaN,NaN
3,2019-03-07,DL,DL,DL,1254,DL,N535US,1254,10397,ATL,...,335.0,337.0,311.0,1.0,2182.0,NaN,NaN,NaN,NaN,NaN
4,2019-03-07,DL,DL,DL,1255,DL,N393DA,1255,14570,RNO,...,89.0,76.0,61.0,1.0,422.0,NaN,NaN,NaN,NaN,NaN


In [193]:
#check values in string columns
object_col = flights_df.select_dtypes('object')
object_col.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,tail_num,origin,origin_city_name,dest,dest_city_name,dup
0,2019-03-07,DL,DL,DL,DL,N344DN,ATL,"Atlanta, GA",MIA,"Miami, FL",N
1,2019-03-07,DL,DL,DL,DL,N314NB,MCI,"Kansas City, MO",LAX,"Los Angeles, CA",N
2,2019-03-07,DL,DL,DL,DL,N332NB,RDU,"Raleigh/Durham, NC",SEA,"Seattle, WA",N
3,2019-03-07,DL,DL,DL,DL,N535US,ATL,"Atlanta, GA",SEA,"Seattle, WA",N
4,2019-03-07,DL,DL,DL,DL,N393DA,RNO,"Reno, NV",SLC,"Salt Lake City, UT",N


In [194]:
#check for unique/duplicate values between columns
flights_df['mkt_unique_carrier'].unique()

array(['DL', 'AA', 'AS', 'UA', 'WN', 'G4', 'NK', 'F9'], dtype=object)

In [195]:
flights_df['branded_code_share'].unique()

array(['DL', 'AA_CODESHARE', 'AA', 'AS_CODESHARE', 'DL_CODESHARE',
       'UA_CODESHARE', 'WN', 'G4', 'NK', 'F9'], dtype=object)

In [196]:
flights_df['mkt_carrier'].unique() 
#shares values with 'mkt_unique_carrier'

array(['DL', 'AA', 'AS', 'UA', 'WN', 'G4', 'NK', 'F9'], dtype=object)

In [197]:
flights_df['op_unique_carrier'].unique()

array(['DL', 'OO', 'AA', 'YX', 'ZW', 'YV', 'KS', 'CP', 'G7', '9E', 'MQ',
       'WN', 'G4', 'NK', 'PT', 'F9'], dtype=object)

In [198]:
flights_df['dup'].unique() #dup only has only one, can drop

array(['N'], dtype=object)

In [199]:
# check rows in mkt_carrier and mkt_unique_carrier for identical values
flights_df.loc[flights_df['mkt_carrier'] != flights_df['mkt_unique_carrier']]
#has no mismatched rows, can drop one

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay


In [200]:
flights_df.drop(['dup', 'mkt_unique_carrier'], axis=1, inplace=True)

In [201]:
flights_df.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019-03-07,DL,DL,1251,DL,N344DN,1251,10397,ATL,"Atlanta, GA",...,109.0,116.0,79.0,1.0,594.0,NaN,NaN,NaN,NaN,NaN
1,2019-03-07,DL,DL,1252,DL,N314NB,1252,13198,MCI,"Kansas City, MO",...,225.0,233.0,218.0,1.0,1363.0,15.0,0.0,8.0,0.0,0.0
2,2019-03-07,DL,DL,1253,DL,N332NB,1253,14492,RDU,"Raleigh/Durham, NC",...,377.0,379.0,344.0,1.0,2354.0,NaN,NaN,NaN,NaN,NaN
3,2019-03-07,DL,DL,1254,DL,N535US,1254,10397,ATL,"Atlanta, GA",...,335.0,337.0,311.0,1.0,2182.0,NaN,NaN,NaN,NaN,NaN
4,2019-03-07,DL,DL,1255,DL,N393DA,1255,14570,RNO,"Reno, NV",...,89.0,76.0,61.0,1.0,422.0,NaN,NaN,NaN,NaN,NaN


In [202]:
#replace NaN in numeric columns
numeric_col = flights_df.select_dtypes(['int64', 'float64'])
numeric_col.head()

,mkt_carrier_fl_num,op_carrier_fl_num,origin_airport_id,dest_airport_id,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,1251,1251,10397,13303,2251,2252.0,1.0,20.0,2312.0,31.0,...,109.0,116.0,79.0,1.0,594.0,NaN,NaN,NaN,NaN,NaN
1,1252,1252,13198,12892,700,715.0,15.0,9.0,724.0,902.0,...,225.0,233.0,218.0,1.0,1363.0,15.0,0.0,8.0,0.0,0.0
2,1253,1253,14492,14747,645,645.0,0.0,26.0,711.0,955.0,...,377.0,379.0,344.0,1.0,2354.0,NaN,NaN,NaN,NaN,NaN
3,1254,1254,10397,14747,1205,1204.0,-1.0,16.0,1220.0,1431.0,...,335.0,337.0,311.0,1.0,2182.0,NaN,NaN,NaN,NaN,NaN
4,1255,1255,14570,14869,1315,1309.0,-6.0,9.0,1318.0,1519.0,...,89.0,76.0,61.0,1.0,422.0,NaN,NaN,NaN,NaN,NaN


In [203]:
float_cols = flights_df.columns[flights_df.dtypes=='float64']
flights_df[float_cols] = flights_df[float_cols].fillna('0.0')
flights_df.fillna(0,inplace=True)

In [204]:
int_cols = flights_df.columns[flights_df.dtypes=='int64']
flights_df[int_cols] = flights_df[int_cols].fillna('0')
flights_df.fillna(0,inplace=True)

In [205]:
flights_df.head()

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2019-03-07,DL,DL,1251,DL,N344DN,1251,10397,ATL,"Atlanta, GA",...,109.0,116,79,1.0,594.0,0.0,0.0,0.0,0.0,0.0
1,2019-03-07,DL,DL,1252,DL,N314NB,1252,13198,MCI,"Kansas City, MO",...,225.0,233,218,1.0,1363.0,15,0,8,0,0
2,2019-03-07,DL,DL,1253,DL,N332NB,1253,14492,RDU,"Raleigh/Durham, NC",...,377.0,379,344,1.0,2354.0,0.0,0.0,0.0,0.0,0.0
3,2019-03-07,DL,DL,1254,DL,N535US,1254,10397,ATL,"Atlanta, GA",...,335.0,337,311,1.0,2182.0,0.0,0.0,0.0,0.0,0.0
4,2019-03-07,DL,DL,1255,DL,N393DA,1255,14570,RNO,"Reno, NV",...,89.0,76,61,1.0,422.0,0.0,0.0,0.0,0.0,0.0


In [206]:
#check rows containing all NaN
flights_df.loc[flights_df.isna().all(1)]

,fl_date,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,...,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay


*convert date/time columns

In [207]:
#convert datetime columns crs_dep_time, dep_time, crs_arr_time, arr_time, wheels_off, wheels_on to datetime format
dt_col = ['crs_dep_time', 'dep_time', 'crs_arr_time', 'arr_time', 'wheels_off', 'wheels_on']

In [208]:
flights_df.loc[:, dt_col].dtypes

crs_dep_time     int64
dep_time        object
crs_arr_time     int64
arr_time        object
wheels_off      object
wheels_on       object
dtype: object

In [209]:
flights_df[dt_col] = flights_df[dt_col].apply(pd.to_numeric, errors='coerce', axis=1)

In [210]:
#function to convert to string
def convert_time(x):
    if np.isnan(x):
        return '00:00'
    
    if x == 2400:
        return '23:59'   
    string = str(int(x))
    
    if len(string)< 4:
        for _ in range(4-len(string)):
            string = '0' + string
    
    return string[:2] + ':' + string[2:]

In [211]:
convert_time(1120)

'11:20'

In [212]:
#run convert_time
flights_df['crs_dep_time'] = flights_df['crs_dep_time'].apply(lambda x: convert_time(x))
flights_df['crs_dep_time'] = pd.to_datetime(flights_df['crs_dep_time'], format = '%H:%M')

flights_df['dep_time'] = flights_df['dep_time'].apply(lambda x: convert_time(x))
flights_df['dep_time'] = pd.to_datetime(flights_df['dep_time'], format = '%H:%M')

flights_df['crs_arr_time'] = flights_df['crs_arr_time'].apply(lambda x: convert_time(x))
flights_df['crs_arr_time'] = pd.to_datetime(flights_df['crs_arr_time'], format = '%H:%M')

flights_df['arr_time'] = flights_df['arr_time'].apply(lambda x: convert_time(x))
flights_df['arr_time'] = pd.to_datetime(flights_df['arr_time'], format = '%H:%M')

flights_df['wheels_off'] = flights_df['wheels_off'].apply(lambda x: convert_time(x))
flights_df['wheels_off'] = pd.to_datetime(flights_df['wheels_off'], format = '%H:%M')

flights_df['wheels_on'] = flights_df['wheels_on'].apply(lambda x: convert_time(x))
flights_df['wheels_on'] = pd.to_datetime(flights_df['wheels_on'], format = '%H:%M')

In [213]:
flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'], format='%Y-%m-%d')

In [214]:
flights_df['fl_date'].dtype

dtype('<M8[ns]')

In [215]:
flights_df.loc[:, dt_col].dtypes

crs_dep_time    datetime64[ns]
dep_time        datetime64[ns]
crs_arr_time    datetime64[ns]
arr_time        datetime64[ns]
wheels_off      datetime64[ns]
wheels_on       datetime64[ns]
dtype: object

In [217]:
#check for missing values again
#check for missing values
total = flights_df.isnull().sum().sort_values(ascending=False)
percent = (flights_df.isnull().sum()/flights_df.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head()

,Total,Percent
late_aircraft_delay,0,0.0
origin,0,0.0
dep_time,0,0.0
crs_dep_time,0,0.0
dest_city_name,0,0.0


In [218]:
#save cleaned data

flights_df.to_csv(path + 'flights_df_clean.csv', index=False)